The aim of this notebook is to demonstrate how to implement logistic regression with stochastic gradient descent. 

In this project, we are trying to do gender voice recognition with logistic regression. The data is obtained from kaggle, where each voice signal has already been analyzed and the data about the voice samples are recorded numerically. 

# Prepare notebook and import data

In [170]:
import pandas as pd
import numpy as np

In [171]:
# import the data and see what is in there (first ten lines)

df_voice=pd.read_csv("voice.csv")
df_voice.shape

(3168, 21)

Taking a look at the dataframe itself:

In [172]:
df_voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


All the variables are represented below:

In [173]:
list(df_voice.columns)

['meanfreq',
 'sd',
 'median',
 'Q25',
 'Q75',
 'IQR',
 'skew',
 'kurt',
 'sp.ent',
 'sfm',
 'mode',
 'centroid',
 'meanfun',
 'minfun',
 'maxfun',
 'meandom',
 'mindom',
 'maxdom',
 'dfrange',
 'modindx',
 'label']

Since label is now a categorical data, split into males and females, it would be beneficial to convert them to 0 and 1, where 0 represent females and 1 represent males

In [174]:
gender_dict = {"male":1, "female":0}
df_voice["gender"] = df_voice["label"].map(gender_dict);
df_voice = df_voice.drop("label",1)

In [175]:
df_voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,1
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,1
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,1


This is the data that we have, and from the data, we observe several problems:

1. The data is listed in such a way that the male voices comes first and female voices comes later, therefore we need to shuffle the rows to ensure that both male and female voices are taken into account of the learning process.
2. Males and females are label, and we need to change them into 0 and 1 so that we can fit the data into the logistic regression algorithm
3. We need a set of testing data to test the algorithm. 

In [176]:
df_voice = df_voice.iloc[np.random.permutation(len(df_voice))]

It is also beneficial to split the data into training and testing data, so that we can verify whether logistic regression can accurately predict the gender of a voice. 

In [177]:
df_test = pd.DataFrame()
df_train = pd.DataFrame()
df_train = df_voice[:2100]
df_test = df_voice[2100:]

In [178]:
df_train.shape

(2100, 21)

In [179]:
# Now we split the testing data to a column vector of gender and
y_train = df_train["gender"].as_matrix()
X_train = df_train.drop("gender",1).as_matrix()

In [180]:
y_train = y_train.transpose()
X_train = X_train.transpose()
X_train = np.append(np.ones((1,2100)), X_train, axis = 0)

In [193]:
y = np.transpose([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
X = 0 0

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [192]:
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

In [186]:
def stochGradDescent(outputVector,inputMatrix,learningSpeed):
    #outputVector should be a binary column vector of classifications
    a = learningSpeed
    y = outputVector
    X = inputMatrix
    numberOfTrials = y.shape[0]
    numberOfEstimators = X.shape[0]
    t = np.ones(numberOfEstimators)/20 #estimator vector (_t_heta)
    for j in range(5):
        for i in range( numberOfTrials ):
            grad = (y[i] - sigmoid(np.dot(np.transpose(t),X[:,i])))
            t += a * grad * X[:,i]
            print grad
    return t

In [187]:
theta = stochGradDescent(y_train,X_train,1.0/2000)

0.220883101509
-0.988044133959
-0.6097521498
0.295399770655
0.266267075136
-0.683621228544
-0.653672512972
-0.712946478825
-0.703662017646
0.329353602246
-0.815327636732
-0.78799074604
0.279501691749
-0.759230324739
-0.609741896793
0.383351840218
0.360555243178
0.380639201617
0.988504913205
-0.999963665116
-0.99631806007
-0.943839027345
-0.99921094723
-0.962236804413
0.0342633981665
-0.964566671293
0.0480371541716
0.0420138884118
-0.999656110903
0.09613635493
-0.943640014418
0.00750680170865
-0.953896211185
0.0693266893994
-0.952827478225
-0.999323887886
0.0132991900818
0.0836272053337
-0.985320265111
0.044951216862
-0.817246219588
0.265986104156
0.0
-0.999921885801
-0.997176661071
0.0163519110876
-0.947791231443
0.124315312607
0.0920067220901
0.0327209339582
-1.0
-0.162213049366
0.515943764364
-0.465103497208
0.51658219371
0.503659147649
-0.447620538695
1.0
-0.998536615239
0.0545502571896
0.00379888810246
0.0
-0.999916354781
-0.88313033437
-0.996497135273
0.088248635432
7.42013023236e

In [158]:
def predict(inputVector,estimatorVector):
    t = estimatorVector
    x = inputVector
    confidence = sigmoid(np.dot(np.transpose(t) , x))
    if confidence > 0.5:
        return 1
    else:
        return 0

0

1